In [74]:
import pandas as pd, numpy as np
#regular expression
import re

MAX_NB_WORDS = 200000
MAX_SEQUENCE_LENGTH = 150

In [75]:
#Load whole yahoo training dataset
unprocessed_training_data = pd.read_csv(r'C:\Users\maximilian.weber\OneDrive - Synpulse\UserRoaming\Desktop\TextClassificationDatasets-20181112T073934Z-001\TextClassificationDatasets\yahoo_answers_csv\train.csv', header=None)
unprocessed_training_data.columns = ['labels','questions_1','questions_2', 'features']

In [76]:
#Load whole yahoo testing dataset
unprocessed_testing_data = pd.read_csv(r'C:\Users\maximilian.weber\OneDrive - Synpulse\UserRoaming\Desktop\TextClassificationDatasets-20181112T073934Z-001\TextClassificationDatasets\yahoo_answers_csv\test.csv', header=None)
unprocessed_testing_data.columns = ['labels','questions_1','questions_2', 'features']

In [77]:
unprocessed_training_data.head()

,labels,questions_1,questions_2,features
0,5,why doesn't an optical mouse work on a glass t...,or even on some surfaces?,Optical mice use an LED and a camera to rapidl...
1,6,What is the best off-road motorcycle trail ?,long-distance trail throughout CA,i hear that the mojave road is amazing!<br />\...
2,3,What is Trans Fat? How to reduce that?,I heard that tras fat is bad for the body. Wh...,Trans fats occur in manufactured foods during ...
3,7,How many planes Fedex has?,I heard that it is the largest airline in the ...,according to the www.fedex.com web site:\nAir ...
4,7,"In the san francisco bay area, does it make se...",the prices of rent and the price of buying doe...,renting vs buying depends on your goals. <br /...


In [78]:
unprocessed_testing_data.head()

,labels,questions_1,questions_2,features
0,9,What makes friendship click?,How does the spark keep going?,good communication is what does it. Can you m...
1,2,Why does Zebras have stripes?,What is the purpose or those stripes? Who do t...,this provides camouflage - predator vision is ...
2,4,What did the itsy bitsy sipder climb up?,NaN,waterspout
3,4,What is the difference between a Bachelors and...,NaN,One difference between a Bachelors and a Maste...
4,3,Why do women get PMS?,NaN,Premenstrual syndrome (PMS) is a group of symp...


In [79]:
print(len(unprocessed_training_data))

1400000


In [80]:
print(len(unprocessed_testing_data))

60000


In [81]:
unprocessed_training_data.isnull().sum()

labels              0
questions_1         0
questions_2    631675
features        24579
dtype: int64

In [82]:
unprocessed_testing_data.isnull().sum()

labels             0
questions_1        0
questions_2    27106
features        1034
dtype: int64

In [83]:
del unprocessed_training_data['questions_1']
del unprocessed_training_data['questions_2']

In [84]:
del unprocessed_testing_data['questions_1']
del unprocessed_testing_data['questions_2']

In [85]:
unprocessed_training_data.head()

,labels,features
0,5,Optical mice use an LED and a camera to rapidl...
1,6,i hear that the mojave road is amazing!<br />\...
2,3,Trans fats occur in manufactured foods during ...
3,7,according to the www.fedex.com web site:\nAir ...
4,7,renting vs buying depends on your goals. <br /...


In [86]:
unprocessed_testing_data.head()

,labels,features
0,9,good communication is what does it. Can you m...
1,2,this provides camouflage - predator vision is ...
2,4,waterspout
3,4,One difference between a Bachelors and a Maste...
4,3,Premenstrual syndrome (PMS) is a group of symp...


In [87]:
processed_training_data = unprocessed_training_data.dropna()
processed_training_data.isnull().sum()

labels      0
features    0
dtype: int64

In [88]:
processed_testing_data = unprocessed_testing_data.dropna()
processed_testing_data.isnull().sum()

labels      0
features    0
dtype: int64

In [89]:
print(len(processed_training_data))

1375421


In [90]:
print(len(processed_testing_data))

58966


In [91]:
#Get data content of training data with label 1-3
training_data_label_1 = processed_training_data[processed_training_data["labels"] == 1]
training_data_label_2 = processed_training_data[processed_training_data["labels"] == 2]
training_data_label_3 = processed_training_data[processed_training_data["labels"] == 3]

In [92]:
##Get data content of testing data with label 1-3
testing_data_label_1 = processed_testing_data[processed_testing_data['labels'] == 1]
testing_data_label_2 = processed_testing_data[processed_testing_data['labels'] == 2]
testing_data_label_3 = processed_testing_data[processed_testing_data['labels'] == 3]

In [93]:
print(len(training_data_label_1))
print(len(training_data_label_2))
print(len(training_data_label_3))

138700
139991
136996


In [94]:
print(len(testing_data_label_1))
print(len(testing_data_label_2))
print(len(testing_data_label_3))

5936
5999
5874


In [95]:
#cut datasets with label 1-3 to size of 10'000
training_data_label_1 = training_data_label_1[:50000]
training_data_label_2 = training_data_label_2[:50000]
training_data_label_3 = training_data_label_3[:50000]

In [96]:
print(len(training_data_label_1))
print(len(training_data_label_2))
print(len(training_data_label_3))

50000
50000
50000


In [97]:
training_data_label_1.head()

,labels,features
61,1,He selects it himself.
64,1,"Al - Albert, Alfred, Alan<br />\nBob - Robert ..."
66,1,There doesn't seem to be a consensus on this. ...
82,1,While historically the use of the semicolon an...
94,1,Fir trees. Their more expensive but it's reall...


In [98]:
#merge training training data label 1 with 2
training_data_label_1_2 = training_data_label_1.append(training_data_label_2)

In [99]:
#merge training data label 1 & 2 with 3
training_data_label_1_2_3 = training_data_label_1_2.append(training_data_label_3)

In [100]:
print(len(training_data_label_1_2_3))

150000


In [101]:
testing_data_label_1.head()

,labels,features
20,1,I think it goes back to the fact you have a pl...
29,1,Saturnalia\n\nhttp://en.wikipedia.org/wiki/Sat...
41,1,No. there is no penetration and you neither s...
45,1,"Kurds (kûrds, kʊrds) , a non-Arab Middle Easte..."
52,1,As diverse as this country is there must be th...


In [102]:
#merge testinig label data
testing_data_label_1_2 = testing_data_label_1.append(testing_data_label_2)

In [103]:
testing_data_label_1_2_3 = testing_data_label_1_2.append(testing_data_label_3)

In [104]:
print(len(testing_data_label_1_2_3))

17809


In [105]:
#shuffle training data
from sklearn.utils import shuffle
shuffled_training_data = shuffle(training_data_label_1_2_3)

In [106]:
#shuffle testing data
shuffled_testing_data = shuffle(testing_data_label_1_2_3)

In [107]:
#fill x_train with training sentences
#fill y_train with corresponding labels
x_train_unordered = shuffled_training_data['features']
y_train_unordered = shuffled_training_data['labels']

In [108]:
print(len(x_train_unordered))
print(len(y_train_unordered))

150000
150000


In [109]:
#fill x_test with testing sentences
#fill y_test with corresponding labels (validation purpose)
x_test_unordered = shuffled_testing_data['features']
y_test_unordered = shuffled_testing_data['labels']

In [110]:
print(len(x_test_unordered))
print(len(y_test_unordered))

17809
17809


In [111]:
x_train = []
y_train = []
for i in y_train_unordered:
    y_train.append(i)
    
for i in x_train_unordered:
    x_train.append(str(i))

y_train = np.array(y_train).reshape(-1,1) #important for keras

In [112]:
x_test = []
y_test = []
for i in y_test_unordered:
    y_test.append(i)
    
for i in x_test_unordered:
    x_test.append(i)

In [113]:
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\maximilian.weber\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [114]:
from nltk.corpus import stopwords

#convert into a list of words
#remove unnecessary split into words, no hyphens
def sentence_to_wordlist(raw):
    """
    Receives a raw review and clean it using the following steps:
    1. Remove all non-words
    2. Transform the review in lower case
    3. Remove all stop words
    4. Perform stemming

    Args:
        review: the review that iwill be cleaned
    Returns:
        a clean review using the mentioned steps above.
    """
    
    clean = re.sub("[^A-Za-z0-9]", " ", str(raw))
    clean = clean.lower()
    review = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    clean = review.sub('', str(clean))
    clean = clean.split()
    
    lemmatizer = WordNetLemmatizer()
    clean = [lemmatizer.lemmatize(i, pos='v') for i in clean]
    return clean

In [43]:
from tqdm import tqdm

#split trainig sentences into words
split_training_sentences = []
for raw_sentence in tqdm(x_train):
    split_training_sentences.append(sentence_to_wordlist(raw_sentence))

100%|██████████| 150000/150000 [03:08<00:00, 795.35it/s] 


In [44]:
print(split_training_sentences[132])

['use', 'get', 'dont', 'think', 'little', 'bleed', 'nail', 'due', 'slight', 'traumas']


In [45]:
#split testing sententes into words
split_testing_sentences = []
for raw in tqdm(x_test):
    split_testing_sentences.append(sentence_to_wordlist(raw))

100%|██████████| 17809/17809 [00:24<00:00, 736.10it/s]


In [46]:
print(split_testing_sentences[4])

['course', 'different', 'ways', 'define', 'worst', 'could', 'property', 'damage', 'loss', 'life', 'powerful', 'focus', 'actual', 'origin', 'quake', 'powerful', 'epicenter', 'place', 'earth', 'directly', 'focus', 'n', 'nprior', 'recent', 'time', 'one', 'accurate', 'instrument', 'measure', 'energy', 'release', 'earthquake', 'one', 'keep', 'accurate', 'record', 'damage', 'longer', 'ago', 'earthquake', 'occur', 'less', 'develop', 'area', 'occur', 'less', 'know', 'information', 'regard', 'shock', 'duration', 'usually', 'available', 'n', 'ngood', 'source', 'information', 'n', 'nhttp', 'wwwneic', 'cr', 'usgs', 'gov', 'neis', 'eqlists', 'eqsmosde', 'html', 'nhttp', 'www', 'usgs', 'gov', 'click', 'earthquake', 'n', 'naccording', 'u', 'geological', 'survey', 'national', 'earthquake', 'information', 'center', 'n', 'nmost', 'destructive', 'know', 'earthquakes', 'record', 'world', 'n', '50', '000', 'deaths', 'n', 'list', 'order', 'greatest', 'number', 'deaths', 'n', 'n', 'ndate', 'location', 'death

In [72]:
#text to numbers ordered by most used 1 to less used high number
from keras.preprocessing.text import Tokenizer

#allows to vectorize a text corpus, by turning each text into either a sequence of integers
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(split_training_sentences)

tokenized_training_set = tokenizer.texts_to_sequences(split_training_sentences)
tokenized_testing_set = tokenizer.texts_to_sequences(split_testing_sentences)
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

In [73]:
import pickle

# saving tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [48]:
#pad every sentence to the same sentence length
from keras.preprocessing.sequence import pad_sequences 

maxlen = MAX_SEQUENCE_LENGTH

padded_training_set = pad_sequences(tokenized_training_set, maxlen=maxlen)
padded_testing_set = pad_sequences(tokenized_testing_set, maxlen=maxlen)

In [49]:
print(padded_training_set[4])

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0    86
    32    12    61  2104   108   912    24  1850 11937     1    66   234
   317   634   348     2    37   910]


In [50]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

In [51]:
embedding_dim = 50
embedding_matrix = create_embedding_matrix(
    r'C:\Users\maximilian.weber\Downloads\glove.6B\glove.6B.50d.txt',
    tokenizer.word_index, embedding_dim)

In [52]:
#necessary if not binary
from keras.utils.np_utils import to_categorical

categorical_labels = to_categorical(y_train)

In [53]:
from keras.models import Sequential
from keras import layers

embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim,
                           weights=[embedding_matrix],
                           input_length=maxlen, 
                           trainable=True))

model.add(layers.SpatialDropout1D(0.5))
model.add(layers.LSTM(196, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Dense(4, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 150, 50)           8699450   
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 150, 50)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               193648    
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 788       
Total params: 8,893,886
Trainable params: 8,893,886
Non-trainable params: 0
_________________________________________________________________


In [54]:
history = model.fit(padded_training_set, categorical_labels, batch_size=64, epochs=3, validation_split=0.1)

Train on 135000 samples, validate on 15000 samples
Epoch 1/3


 12928/135000 [=>............................] - ETA: 2:05:03 - loss: 1.3958 - acc: 0.20 - ETA: 1:12:47 - loss: 1.3564 - acc: 0.29 - ETA: 54:56 - loss: 1.3212 - acc: 0.3490 - ETA: 45:47 - loss: 1.3039 - acc: 0.35 - ETA: 40:12 - loss: 1.2778 - acc: 0.38 - ETA: 36:48 - loss: 1.2605 - acc: 0.38 - ETA: 38:41 - loss: 1.2395 - acc: 0.40 - ETA: 39:28 - loss: 1.2327 - acc: 0.40 - ETA: 38:16 - loss: 1.2145 - acc: 0.41 - ETA: 37:10 - loss: 1.1971 - acc: 0.43 - ETA: 36:08 - loss: 1.1859 - acc: 0.43 - ETA: 35:17 - loss: 1.1773 - acc: 0.44 - ETA: 34:43 - loss: 1.1652 - acc: 0.46 - ETA: 33:47 - loss: 1.1529 - acc: 0.46 - ETA: 32:48 - loss: 1.1409 - acc: 0.47 - ETA: 31:58 - loss: 1.1303 - acc: 0.48 - ETA: 31:12 - loss: 1.1234 - acc: 0.48 - ETA: 30:31 - loss: 1.1192 - acc: 0.48 - ETA: 29:55 - loss: 1.1133 - acc: 0.48 - ETA: 29:21 - loss: 1.1061 - acc: 0.48 - ETA: 28:51 - loss: 1.0981 - acc: 0.49 - ETA: 28:26 - loss: 1.0895 - acc: 0.49 - ETA: 28:13 - loss: 1.0842 - acc: 0.49 - ETA: 28:15 - loss: 1.0844

 25984/135000 [====>.........................] - ETA: 18:20 - loss: 0.8355 - acc: 0.62 - ETA: 18:19 - loss: 0.8344 - acc: 0.62 - ETA: 18:18 - loss: 0.8344 - acc: 0.62 - ETA: 18:17 - loss: 0.8345 - acc: 0.62 - ETA: 18:16 - loss: 0.8339 - acc: 0.62 - ETA: 18:15 - loss: 0.8335 - acc: 0.62 - ETA: 18:14 - loss: 0.8336 - acc: 0.62 - ETA: 18:13 - loss: 0.8332 - acc: 0.62 - ETA: 18:12 - loss: 0.8321 - acc: 0.62 - ETA: 18:11 - loss: 0.8317 - acc: 0.62 - ETA: 18:10 - loss: 0.8320 - acc: 0.62 - ETA: 18:09 - loss: 0.8320 - acc: 0.62 - ETA: 18:08 - loss: 0.8313 - acc: 0.62 - ETA: 18:07 - loss: 0.8311 - acc: 0.62 - ETA: 18:06 - loss: 0.8308 - acc: 0.62 - ETA: 18:05 - loss: 0.8305 - acc: 0.62 - ETA: 18:04 - loss: 0.8306 - acc: 0.62 - ETA: 18:03 - loss: 0.8300 - acc: 0.62 - ETA: 18:02 - loss: 0.8304 - acc: 0.62 - ETA: 18:01 - loss: 0.8300 - acc: 0.62 - ETA: 18:00 - loss: 0.8297 - acc: 0.62 - ETA: 17:59 - loss: 0.8293 - acc: 0.62 - ETA: 17:58 - loss: 0.8289 - acc: 0.62 - ETA: 17:57 - loss: 0.8284 - acc

 39040/135000 [=======>......................] - ETA: 17:18 - loss: 0.7821 - acc: 0.65 - ETA: 17:17 - loss: 0.7815 - acc: 0.65 - ETA: 17:17 - loss: 0.7811 - acc: 0.65 - ETA: 17:16 - loss: 0.7807 - acc: 0.65 - ETA: 17:15 - loss: 0.7806 - acc: 0.65 - ETA: 17:15 - loss: 0.7804 - acc: 0.65 - ETA: 17:14 - loss: 0.7803 - acc: 0.65 - ETA: 17:13 - loss: 0.7799 - acc: 0.65 - ETA: 17:12 - loss: 0.7801 - acc: 0.65 - ETA: 17:12 - loss: 0.7797 - acc: 0.65 - ETA: 17:11 - loss: 0.7795 - acc: 0.65 - ETA: 17:10 - loss: 0.7794 - acc: 0.65 - ETA: 17:09 - loss: 0.7794 - acc: 0.65 - ETA: 17:09 - loss: 0.7790 - acc: 0.65 - ETA: 17:08 - loss: 0.7788 - acc: 0.65 - ETA: 17:07 - loss: 0.7787 - acc: 0.65 - ETA: 17:06 - loss: 0.7784 - acc: 0.65 - ETA: 17:06 - loss: 0.7783 - acc: 0.65 - ETA: 17:05 - loss: 0.7779 - acc: 0.65 - ETA: 17:04 - loss: 0.7778 - acc: 0.65 - ETA: 17:03 - loss: 0.7775 - acc: 0.65 - ETA: 17:03 - loss: 0.7779 - acc: 0.65 - ETA: 17:02 - loss: 0.7779 - acc: 0.65 - ETA: 17:01 - loss: 0.7776 - acc

 52096/135000 [==========>...................] - ETA: 14:48 - loss: 0.7477 - acc: 0.67 - ETA: 14:48 - loss: 0.7475 - acc: 0.67 - ETA: 14:47 - loss: 0.7472 - acc: 0.67 - ETA: 14:46 - loss: 0.7469 - acc: 0.67 - ETA: 14:46 - loss: 0.7466 - acc: 0.67 - ETA: 14:45 - loss: 0.7465 - acc: 0.67 - ETA: 14:45 - loss: 0.7462 - acc: 0.67 - ETA: 14:44 - loss: 0.7459 - acc: 0.67 - ETA: 14:43 - loss: 0.7460 - acc: 0.67 - ETA: 14:42 - loss: 0.7459 - acc: 0.67 - ETA: 14:42 - loss: 0.7455 - acc: 0.67 - ETA: 14:41 - loss: 0.7454 - acc: 0.67 - ETA: 14:40 - loss: 0.7451 - acc: 0.67 - ETA: 14:39 - loss: 0.7448 - acc: 0.67 - ETA: 14:39 - loss: 0.7449 - acc: 0.67 - ETA: 14:38 - loss: 0.7446 - acc: 0.67 - ETA: 14:38 - loss: 0.7445 - acc: 0.67 - ETA: 14:38 - loss: 0.7445 - acc: 0.67 - ETA: 14:37 - loss: 0.7445 - acc: 0.67 - ETA: 14:37 - loss: 0.7442 - acc: 0.67 - ETA: 14:37 - loss: 0.7440 - acc: 0.67 - ETA: 14:37 - loss: 0.7437 - acc: 0.67 - ETA: 14:37 - loss: 0.7437 - acc: 0.67 - ETA: 14:37 - loss: 0.7436 - acc

...

 91392/135000 [===================>..........] - ETA: 9:34 - loss: 0.5023 - acc: 0.797 - ETA: 9:35 - loss: 0.5022 - acc: 0.797 - ETA: 9:35 - loss: 0.5022 - acc: 0.797 - ETA: 9:34 - loss: 0.5022 - acc: 0.797 - ETA: 9:34 - loss: 0.5022 - acc: 0.797 - ETA: 9:33 - loss: 0.5022 - acc: 0.797 - ETA: 9:32 - loss: 0.5022 - acc: 0.797 - ETA: 9:32 - loss: 0.5022 - acc: 0.797 - ETA: 9:31 - loss: 0.5022 - acc: 0.797 - ETA: 9:30 - loss: 0.5023 - acc: 0.797 - ETA: 9:29 - loss: 0.5022 - acc: 0.797 - ETA: 9:29 - loss: 0.5022 - acc: 0.797 - ETA: 9:28 - loss: 0.5023 - acc: 0.797 - ETA: 9:27 - loss: 0.5022 - acc: 0.797 - ETA: 9:27 - loss: 0.5023 - acc: 0.797 - ETA: 9:26 - loss: 0.5022 - acc: 0.797 - ETA: 9:25 - loss: 0.5022 - acc: 0.797 - ETA: 9:24 - loss: 0.5022 - acc: 0.797 - ETA: 9:24 - loss: 0.5021 - acc: 0.797 - ETA: 9:23 - loss: 0.5021 - acc: 0.797 - ETA: 9:22 - loss: 0.5021 - acc: 0.797 - ETA: 9:22 - loss: 0.5020 - acc: 0.797 - ETA: 9:21 - loss: 0.5020 - acc: 0.797 - ETA: 9:20 - loss: 0.5020 - acc:

104448/135000 [======================>.......] - ETA: 7:33 - loss: 0.5025 - acc: 0.797 - ETA: 7:33 - loss: 0.5026 - acc: 0.797 - ETA: 7:32 - loss: 0.5025 - acc: 0.797 - ETA: 7:32 - loss: 0.5024 - acc: 0.797 - ETA: 7:32 - loss: 0.5025 - acc: 0.797 - ETA: 7:31 - loss: 0.5025 - acc: 0.797 - ETA: 7:31 - loss: 0.5025 - acc: 0.797 - ETA: 7:30 - loss: 0.5024 - acc: 0.797 - ETA: 7:30 - loss: 0.5024 - acc: 0.797 - ETA: 7:29 - loss: 0.5023 - acc: 0.797 - ETA: 7:28 - loss: 0.5022 - acc: 0.797 - ETA: 7:28 - loss: 0.5023 - acc: 0.797 - ETA: 7:27 - loss: 0.5024 - acc: 0.797 - ETA: 7:27 - loss: 0.5023 - acc: 0.797 - ETA: 7:26 - loss: 0.5024 - acc: 0.797 - ETA: 7:26 - loss: 0.5023 - acc: 0.797 - ETA: 7:25 - loss: 0.5023 - acc: 0.797 - ETA: 7:25 - loss: 0.5024 - acc: 0.797 - ETA: 7:24 - loss: 0.5024 - acc: 0.797 - ETA: 7:23 - loss: 0.5024 - acc: 0.797 - ETA: 7:23 - loss: 0.5025 - acc: 0.797 - ETA: 7:22 - loss: 0.5024 - acc: 0.797 - ETA: 7:21 - loss: 0.5025 - acc: 0.797 - ETA: 7:21 - loss: 0.5025 - acc:

117504/135000 [=========================>....] - ETA: 5:31 - loss: 0.5012 - acc: 0.798 - ETA: 5:30 - loss: 0.5011 - acc: 0.798 - ETA: 5:30 - loss: 0.5010 - acc: 0.798 - ETA: 5:29 - loss: 0.5010 - acc: 0.798 - ETA: 5:28 - loss: 0.5010 - acc: 0.798 - ETA: 5:27 - loss: 0.5009 - acc: 0.798 - ETA: 5:27 - loss: 0.5009 - acc: 0.798 - ETA: 5:26 - loss: 0.5009 - acc: 0.798 - ETA: 5:25 - loss: 0.5008 - acc: 0.798 - ETA: 5:25 - loss: 0.5008 - acc: 0.798 - ETA: 5:24 - loss: 0.5007 - acc: 0.798 - ETA: 5:23 - loss: 0.5005 - acc: 0.798 - ETA: 5:22 - loss: 0.5005 - acc: 0.798 - ETA: 5:22 - loss: 0.5005 - acc: 0.798 - ETA: 5:21 - loss: 0.5004 - acc: 0.798 - ETA: 5:20 - loss: 0.5004 - acc: 0.798 - ETA: 5:20 - loss: 0.5005 - acc: 0.798 - ETA: 5:19 - loss: 0.5005 - acc: 0.798 - ETA: 5:18 - loss: 0.5005 - acc: 0.798 - ETA: 5:18 - loss: 0.5004 - acc: 0.798 - ETA: 5:17 - loss: 0.5005 - acc: 0.798 - ETA: 5:16 - loss: 0.5004 - acc: 0.798 - ETA: 5:16 - loss: 0.5003 - acc: 0.798 - ETA: 5:15 - loss: 0.5003 - acc:

130624/135000 [============================>.] - ETA: 3:12 - loss: 0.4999 - acc: 0.798 - ETA: 3:11 - loss: 0.4999 - acc: 0.798 - ETA: 3:10 - loss: 0.4999 - acc: 0.798 - ETA: 3:09 - loss: 0.4998 - acc: 0.798 - ETA: 3:09 - loss: 0.4998 - acc: 0.798 - ETA: 3:08 - loss: 0.4998 - acc: 0.798 - ETA: 3:08 - loss: 0.4997 - acc: 0.798 - ETA: 3:07 - loss: 0.4997 - acc: 0.798 - ETA: 3:06 - loss: 0.4996 - acc: 0.798 - ETA: 3:06 - loss: 0.4996 - acc: 0.798 - ETA: 3:05 - loss: 0.4996 - acc: 0.798 - ETA: 3:04 - loss: 0.4995 - acc: 0.798 - ETA: 3:04 - loss: 0.4995 - acc: 0.798 - ETA: 3:03 - loss: 0.4995 - acc: 0.798 - ETA: 3:02 - loss: 0.4995 - acc: 0.798 - ETA: 3:02 - loss: 0.4995 - acc: 0.798 - ETA: 3:01 - loss: 0.4995 - acc: 0.798 - ETA: 3:00 - loss: 0.4995 - acc: 0.798 - ETA: 3:00 - loss: 0.4995 - acc: 0.798 - ETA: 2:59 - loss: 0.4995 - acc: 0.798 - ETA: 2:58 - loss: 0.4995 - acc: 0.798 - ETA: 2:58 - loss: 0.4995 - acc: 0.798 - ETA: 2:57 - loss: 0.4994 - acc: 0.798 - ETA: 2:56 - loss: 0.4994 - acc:

135000/135000 [==============================] - ETA: 48s - loss: 0.4994 - acc: 0.79 - ETA: 47s - loss: 0.4994 - acc: 0.79 - ETA: 46s - loss: 0.4993 - acc: 0.79 - ETA: 46s - loss: 0.4993 - acc: 0.79 - ETA: 45s - loss: 0.4994 - acc: 0.79 - ETA: 44s - loss: 0.4993 - acc: 0.79 - ETA: 43s - loss: 0.4992 - acc: 0.79 - ETA: 43s - loss: 0.4993 - acc: 0.79 - ETA: 42s - loss: 0.4993 - acc: 0.79 - ETA: 41s - loss: 0.4993 - acc: 0.79 - ETA: 41s - loss: 0.4993 - acc: 0.79 - ETA: 40s - loss: 0.4993 - acc: 0.79 - ETA: 39s - loss: 0.4992 - acc: 0.79 - ETA: 38s - loss: 0.4992 - acc: 0.79 - ETA: 38s - loss: 0.4991 - acc: 0.79 - ETA: 37s - loss: 0.4991 - acc: 0.79 - ETA: 36s - loss: 0.4991 - acc: 0.79 - ETA: 36s - loss: 0.4991 - acc: 0.79 - ETA: 35s - loss: 0.4991 - acc: 0.79 - ETA: 34s - loss: 0.4991 - acc: 0.79 - ETA: 33s - loss: 0.4991 - acc: 0.79 - ETA: 33s - loss: 0.4990 - acc: 0.79 - ETA: 32s - loss: 0.4990 - acc: 0.79 - ETA: 31s - loss: 0.4990 - acc: 0.79 - ETA: 31s - loss: 0.4990 - acc: 0.79 - E

In [59]:
results = model.predict(padded_testing_set)

In [69]:
# Save the weights
model.save_weights('model_weights.h5')

# Save the model architecture
with open('model_architecture.json', 'w') as f:
    f.write(model.to_json())

In [67]:
number = 501

probability = max(results[number])
predicted_category = np.argmax(results[number])
true_category = y_test[number]
text_sequence = split_testing_sentences[number]

print('I\'m ' + str(round(probability * 100)) + '% sure about my prediction!')
print(100*'_')
if predicted_category == 1:
    print('predicted: Society & Culture')
elif predicted_category == 2:
    print('predicted: Science & Math')
elif predicted_category == 3:
    print('predicted: Health')
print(100*'_')
if true_category == 1:
    print('True Category: Society & Culture')
elif true_category == 2:
    print('True Category: Science & Math')
elif true_category == 3:
    print('True Category: Health')
print(100*'_')
print(text_sequence)
print(100*'_')
print(x_test[number])

I'm 95.0% sure about my prediction!
____________________________________________________________________________________________________
predicted: Health
____________________________________________________________________________________________________
True Category: Health
____________________________________________________________________________________________________
['aspirin', 'blood', 'thinner', 'gastro', 'intestinal', 'problems', 'hemmorhoids', 'could']
____________________________________________________________________________________________________
Not by itself.  Aspirin is a blood thinner, so if you have other gastro-intestinal problems or hemmorhoids, it could.


In [57]:
#gets all maximum numbers of 3 predictions per array
new_list = []
for i in results:
    new_list.append(max(i))

In [58]:
#gets all categories corresponding to each array
categories_list = []
for i in results:
    categories_list.append(np.argmax(i))

In [115]:
#amount of overconfident predictions
overconfident_amount = 0
for i, j, k in zip(new_list, y_test, categories_list):
    if i >= 0.9 and j != k:
        overconfident_amount = overconfident_amount + 1
    else:
        pass

In [116]:
#go through all overconfident predicted ones and give back index in list
lookagain = []
for i, j, k in zip(new_list, y_test, categories_list):
    if i >= 0.9 and j != k:
        lookagain.append(new_list.index(i))
    else:
        pass

In [117]:
print(lookagain[449])

17791


In [109]:
##NOTWORKING FOR NOW!!!!
listofallwrong = []
number = 0
for i in split_testing_sentences:
    if split_testing_sentences.index(i) == lookagain[number]:
        listofallwrong.append(i)
        number = number + 1
    else:
        pass

In [118]:
percent = round(overconfident_amount/len(results) * 100,2)
print(str(percent)+'%')
print(overconfident_amount)
print(len(results))

#vorher 3% jetzt 1.56% mit glove trainable = false

2.53%
450
17809


In [375]:
good = 0
bad = 0
for x, y in zip(results, y_test):
    true_category = y
    probability = max(x)
    predicted_category = np.argmax(x)

    if predicted_category == true_category:
        good = good + 1
    else:
        bad = bad + 1

In [376]:
print(good)
print(bad)
ow = bad + good
print(str(round(good/ow*100))+'%')

14199
3610
80%


In [61]:
example = input('Test: ')
example = sentence_to_wordlist(example)
example = tokenizer.texts_to_sequences([example])
example = pad_sequences(example, maxlen=maxlen)

category_probability = model.predict(example)

print(max(category_probability[0]))

number = np.argmax(category_probability)
if number == 1:
    print('Society & Culture')
elif number == 2:
    print('Science & Math')
elif number == 3:
    print('Health')

Test: Ongoing climate change is thought to disrupt trophic relationships, with consequences for complex interspecific interactions, yet the effects of climate change on species interactions are poorly understood, and such effects have not been documented at a global scale. Using a single database of 38,191 nests from 237 populations, we found that shorebirds have experienced a worldwide increase in nest predation over the past 70 years. Historically, there existed a latitudinal gradient in nest predation, with the highest rates in the tropics; however, this pattern has been recently reversed in the Northern Hemisphere, most notably in the Arctic. This increased nest predation is consistent with climate-induced shifts in predator-prey relationships.
0.98292774
Science & Math
